# LSTM, GRU, 1D CNN MODEL 비교 

In [23]:
# GPU로 Tesla T4가 준비돼있습니다.
!nvidia-smi

Wed Jun 26 01:28:04 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.239.06   Driver Version: 470.239.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0    34W /  70W |   1244MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [24]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import json

# 1. 데이터 가져오기

In [25]:
train_data_path ="~/aiffel/dktc/data/train.csv"
data = pd.read_csv(train_data_path)
data.head()

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [26]:
data.shape

(3950, 3)

# 2. 데이터 전처리

### 2-1)결측치, 중복데이터 확인

In [27]:
print(data.isna().sum())

idx             0
class           0
conversation    0
dtype: int64


In [28]:
#중복인 행 확인
data['duplicates'] = data.duplicated(['conversation'])
data[data['duplicates'] == True]

,idx,class,conversation,duplicates
392,392,기타 괴롭힘 대화,병신이 아이스크림 먹게 돼 있냐?\n난 먹으면 안 돼? 그만 좀 해.\n당연히 안 ...,True
523,523,직장 내 괴롭힘 대화,과장님. 저 이번에 휴가 좀 갔다와도 되겠습니까.?\n휴가? 왜??\n좀 쉬다가 오...,True
789,789,기타 괴롭힘 대화,야 얘 이 쪽 손가락 세 개밖에 없다\n엥 손가락이 세개밖에 없을 수가 있어?\n봐...,True
824,824,기타 괴롭힘 대화,이야 내동생 여자친구한테 편지쓰네?\n 아 형 돌려줘.\n 어디보자. 사랑하는 여친...,True
869,869,직장 내 괴롭힘 대화,지현씨 나 소개팅 좀 시켜줘봐\n네? 저 주변에 아는 사람이 없어서요\n아 상사라 ...,True
...,...,...,...,...
3797,3797,기타 괴롭힘 대화,너 이번 방학 때 쌍꺼풀 수술 하고왔지?\n아닌데?\n아니긴 뭐가 아니야. 눈이 이...,True
3798,3798,기타 괴롭힘 대화,안녕하세요 지금 먹방 촬영중인데 촬영가능할까요?\n안돼요\n한번만 안될까요?\n안돼...,True
3855,3855,기타 괴롭힘 대화,그 소문 진짜야? 너가 다른 애들 뒷담화하고 다녔다며?\n응? 나 그런 적 없는데?...,True
3874,3874,기타 괴롭힘 대화,야 니 왤캐 못생겼냐?\n뭐라그랬냐?\n으 나 보고 말하지마 니 얼굴보면 토나올거 ...,True


In [29]:
#중복제거
data = data.drop_duplicates(['conversation'])

In [30]:
#conversation과 class만 남김
data = data[['conversation','class']]
data.head()

,conversation,class
0,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...,협박 대화
1,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...,협박 대화
2,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...,기타 괴롭힘 대화
3,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...,갈취 대화
4,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...,갈취 대화


In [31]:
data.shape

(3846, 2)

### 2-2)레이블인코딩

In [32]:
CLASS_NAMES = ['협박 대화', '갈취 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화']

encoder = LabelEncoder()
encoder.fit(CLASS_NAMES)

data['class'] = encoder.transform(data['class'])

corpus = data["conversation"]
data.head()

,conversation,class
0,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...,3
1,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...,3
2,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...,1
3,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...,0
4,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...,0


### 2-3)토크나이저 및 단어사전 생성

In [33]:
def tokenize(corpus):
    #토크나이저 객체생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=1000, 
        filters=' ',
        oov_token="<unk>")
    
    tokenizer.fit_on_texts(corpus) #corpus 기준으로 단어 사전 생성
    tensor = tokenizer.texts_to_sequences(corpus) #정수시퀀스
    
    #시퀀스패딩
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, 
                                                           padding='post',
                                                           maxlen=20) #최대길이
    print(tensor,tokenizer)
    return tensor, tokenizer

#데이터셋에서 토큰화 및 패딩작업 결과를 저장
tensor, tokenizer = tokenize(corpus)

[[  1 210   1 ...   1  42   1]
 [  1   1   1 ...   1   1 401]
 [  1  97   1 ...   1 136   1]
 ...
 [495   3 110 ...  27   1   1]
 [  1   1 151 ... 348   1   1]
 [  1   1   1 ... 159 113   1]] <keras_preprocessing.text.Tokenizer object at 0x7fcb628d2250>


### 2-4)데이터 분할

In [34]:
#Train data와 Test data 분할
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tensor, data['class'], 
                                                    test_size=0.1, 
                                                    stratify = data['class'], 
                                                    random_state=42)

# 3. 모델생성 및 훈련

## 3-1) 모델 구성

In [35]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,SimpleRNN,Dropout, LSTM, GRU, Conv1D, GlobalMaxPooling1D, Dense
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

vocab_size = 1000  # 어휘 사전의 크기
word_vector_dim = 20  # 단어 하나를 표현하는 임베딩 벡터의 차원수

#RNN 모델 생성
def create_rnn_model(vocab_size, word_vector_dim):
    model = Sequential()
    model.add(Embedding(vocab_size, word_vector_dim, input_length=None))
    model.add(SimpleRNN(256)) 
    model.add(Dropout(0.2)) 
    model.add(Dense(64, activation='relu'))
    model.add(Dense(4, activation='softmax')) 
    return model

#LSTM 모델생성
def create_lstm_model(vocab_size, word_vector_dim):
    model = Sequential()
    model.add(Embedding(vocab_size, word_vector_dim, input_length=None))
    model.add(LSTM(256)) 
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(4, activation='softmax'))
    return model

#GRU 모델생성
def create_gru_model(vocab_size, word_vector_dim):
    model = Sequential()
    model.add(Embedding(vocab_size, word_vector_dim, input_length=None))
    model.add(GRU(256)) 
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(4, activation='softmax'))
    return model

#CNN 모델생성
def create_1d_cnn_model(vocab_size, word_vector_dim):
    model = Sequential()
    model.add(Embedding(vocab_size, word_vector_dim, input_length=None))
    model.add(Conv1D(256, 3, activation='relu')) 
    model.add(GlobalMaxPooling1D())
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(4, activation='softmax'))  
    return model

In [36]:
#콜백함수
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',  # 검증 손실을 기준으로 조기 종료 여부 판단
    patience=5,         # 검증 손실이 개선되지 않는 epoch를 5회까지 허용
    restore_best_weights=True  # 가장 좋은 검증 손실을 낸 모델의 가중치 복원
)

In [37]:
#모델 함수 배열로 가져오김
models = [create_rnn_model(vocab_size, word_vector_dim),
          create_lstm_model(vocab_size, word_vector_dim), 
          create_gru_model(vocab_size, word_vector_dim), 
          create_1d_cnn_model(vocab_size, word_vector_dim)]
model_names = ['RNN', 'LSTM', 'GRU', '1D CNN']

## 3-2) 모델학습 및 평가

In [38]:
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.metrics import classification_report


#kfold 객체생성
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 결과 저장 딕셔러니
results = {}

# 랜덤 시드 리스트 생성
random_seeds = random.sample(range(1, 1000), 5)

#모든 모델 학습 및 평가
for model, model_name in zip(models,model_names):
    #모델별 결과 저장 딕셔너리
    results[model_name] = {}
    
    for seed in random_seeds:
        #랜덤 시드 설정
        np.random.seed(seed)
        tf.random.set_seed(seed)
        
        #fold별 loss, accuracy, f1 score기록
        fold_losses = []
        fold_accuracies = []
        fold_recall = []
        fold_precision = []
        fold_f1s = []

        #K겹 교차검증
        for train_index, val_index in skf.split(X_train, y_train):

            #데이터 분할
            X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
            y_train_fold, y_val_fold = y_train.values[train_index], y_train.values[val_index]

            model.compile(optimizer='adam', 
                          loss='sparse_categorical_crossentropy', 
                          metrics=['accuracy'])

            history = model.fit(X_train_fold, y_train_fold, epochs=50, 
                                validation_data=(X_val_fold, y_val_fold), 
                                callbacks = [early_stopping])

            #모델 평가
            loss, accuracy = model.evaluate(X_test, y_test)
            
            

            #flod평가 loss, accuracy를 배열에 기록
            fold_losses.append(loss)
            fold_accuracies.append(accuracy)
            
            #모델예측
            y_pred = model.predict(X_test)
            y_pred_classes = y_pred.argmax(axis=-1) # 예측 확률 값을 클래스 레이블로 변환
            
            report_dict = classification_report(y_test, y_pred_classes, output_dict=True)

            # 필요한 precision/recall/f1 추출 및 출력
            precision = report_dict['macro avg']['precision']
            recall = report_dict['macro avg']['recall']
            f1 = report_dict['macro avg']['f1-score']

            
            fold_recall.append(precision)
            fold_precision.append(recall)
            fold_f1s.append(f1)
            
        
        #k-fold 평균계산
        avg_loss = np.mean(fold_losses)
        avg_accuracy = np.mean(fold_accuracies)
        avg_recall = np.mean(fold_recall)
        avg_precision = np.mean(fold_precision)
        avg_f1 = np.mean(fold_f1s)
        
        #시드별 결과를 딕셔너리 형태로 저장
        results[model_name][seed] = {
            'loss' : avg_loss,
            'accuracy' : avg_accuracy,
            'recall' : avg_recall,
            'precision' : avg_precision,
            'f1' : avg_f1    
        }

Epoch 1/50
87/87 [==============================] - 3s 24ms/step - loss: 1.3930 - accuracy: 0.2388 - val_loss: 1.3936 - val_accuracy: 0.2511
Epoch 2/50
87/87 [==============================] - 2s 21ms/step - loss: 1.3583 - accuracy: 0.3186 - val_loss: 1.3863 - val_accuracy: 0.2857
Epoch 3/50
87/87 [==============================] - 2s 21ms/step - loss: 1.2913 - accuracy: 0.3815 - val_loss: 1.3711 - val_accuracy: 0.3016
Epoch 4/50
87/87 [==============================] - 2s 22ms/step - loss: 1.1910 - accuracy: 0.4722 - val_loss: 1.4019 - val_accuracy: 0.3377
Epoch 5/50
87/87 [==============================] - 2s 21ms/step - loss: 1.0359 - accuracy: 0.5578 - val_loss: 1.3859 - val_accuracy: 0.3781
Epoch 6/50
87/87 [==============================] - 2s 22ms/step - loss: 0.9235 - accuracy: 0.6254 - val_loss: 1.3800 - val_accuracy: 0.4127
Epoch 7/50
87/87 [==============================] - 2s 22ms/step - loss: 0.7680 - accuracy: 0.6904 - val_loss: 1.4673 - val_accuracy: 0.4343
Epoch 8/50
13

13/13 [==============================] - 0s 3ms/step - loss: 1.5831 - accuracy: 0.4857
Epoch 1/50
87/87 [==============================] - 3s 23ms/step - loss: 0.4414 - accuracy: 0.8371 - val_loss: 0.3240 - val_accuracy: 0.8858
Epoch 2/50
87/87 [==============================] - 2s 21ms/step - loss: 0.3602 - accuracy: 0.8675 - val_loss: 0.4025 - val_accuracy: 0.8584
Epoch 3/50
87/87 [==============================] - 2s 20ms/step - loss: 0.3094 - accuracy: 0.8862 - val_loss: 0.5188 - val_accuracy: 0.8165
Epoch 4/50
87/87 [==============================] - 2s 20ms/step - loss: 0.2549 - accuracy: 0.9104 - val_loss: 0.4984 - val_accuracy: 0.8150
Epoch 5/50
87/87 [==============================] - 2s 20ms/step - loss: 0.2015 - accuracy: 0.9303 - val_loss: 0.7922 - val_accuracy: 0.7197
Epoch 6/50
13/13 [==============================] - 0s 4ms/step - loss: 1.7743 - accuracy: 0.4364
Epoch 1/50
87/87 [==============================] - 3s 22ms/step - loss: 0.4053 - accuracy: 0.8552 - val_loss:

87/87 [==============================] - 2s 21ms/step - loss: 0.1560 - accuracy: 0.9440 - val_loss: 0.3435 - val_accuracy: 0.8815
Epoch 5/50
87/87 [==============================] - 2s 22ms/step - loss: 0.1004 - accuracy: 0.9664 - val_loss: 0.3312 - val_accuracy: 0.8815
Epoch 6/50
13/13 [==============================] - 0s 4ms/step - loss: 2.1581 - accuracy: 0.4416
Epoch 1/50
87/87 [==============================] - 3s 25ms/step - loss: 0.2133 - accuracy: 0.9274 - val_loss: 0.1180 - val_accuracy: 0.9639
Epoch 2/50
87/87 [==============================] - 2s 22ms/step - loss: 0.1418 - accuracy: 0.9538 - val_loss: 0.1982 - val_accuracy: 0.9277
Epoch 3/50
87/87 [==============================] - 2s 23ms/step - loss: 0.1264 - accuracy: 0.9559 - val_loss: 0.2652 - val_accuracy: 0.9104
Epoch 4/50
87/87 [==============================] - 2s 22ms/step - loss: 0.1379 - accuracy: 0.9484 - val_loss: 0.3153 - val_accuracy: 0.8902
Epoch 5/50
87/87 [==============================] - 2s 22ms/step - 

87/87 [==============================] - 2s 21ms/step - loss: 0.0739 - accuracy: 0.9758 - val_loss: 0.1852 - val_accuracy: 0.9263
Epoch 6/50
13/13 [==============================] - 0s 4ms/step - loss: 3.1108 - accuracy: 0.4416
Epoch 1/50
87/87 [==============================] - 2s 9ms/step - loss: 1.3748 - accuracy: 0.2778 - val_loss: 1.3844 - val_accuracy: 0.3680
Epoch 2/50
87/87 [==============================] - 0s 5ms/step - loss: 1.2136 - accuracy: 0.4147 - val_loss: 1.1484 - val_accuracy: 0.4286
Epoch 3/50
87/87 [==============================] - 0s 5ms/step - loss: 1.0427 - accuracy: 0.5134 - val_loss: 1.1122 - val_accuracy: 0.4906
Epoch 4/50
87/87 [==============================] - 0s 5ms/step - loss: 0.9613 - accuracy: 0.5549 - val_loss: 1.1234 - val_accuracy: 0.5108
Epoch 5/50
87/87 [==============================] - 0s 5ms/step - loss: 0.8539 - accuracy: 0.6402 - val_loss: 1.1551 - val_accuracy: 0.5642
Epoch 6/50
87/87 [==============================] - 0s 5ms/step - loss: 

87/87 [==============================] - 0s 5ms/step - loss: 0.3945 - accuracy: 0.8537 - val_loss: 0.6304 - val_accuracy: 0.7572
Epoch 5/50
87/87 [==============================] - 0s 5ms/step - loss: 0.3558 - accuracy: 0.8743 - val_loss: 0.7030 - val_accuracy: 0.7486
Epoch 6/50
13/13 [==============================] - 0s 3ms/step - loss: 1.0412 - accuracy: 0.6078
Epoch 1/50
87/87 [==============================] - 2s 9ms/step - loss: 0.4900 - accuracy: 0.8263 - val_loss: 0.4218 - val_accuracy: 0.8324
Epoch 2/50
87/87 [==============================] - 0s 6ms/step - loss: 0.4302 - accuracy: 0.8483 - val_loss: 0.4964 - val_accuracy: 0.8078
Epoch 3/50
87/87 [==============================] - 0s 5ms/step - loss: 0.3869 - accuracy: 0.8595 - val_loss: 0.5255 - val_accuracy: 0.7876
Epoch 4/50
87/87 [==============================] - 0s 5ms/step - loss: 0.3557 - accuracy: 0.8725 - val_loss: 0.6762 - val_accuracy: 0.7442
Epoch 5/50
87/87 [==============================] - 0s 5ms/step - loss: 0

...

87/87 [==============================] - 0s 5ms/step - loss: 0.4286 - accuracy: 0.8397 - val_loss: 0.6553 - val_accuracy: 0.7558
Epoch 4/50
87/87 [==============================] - 0s 5ms/step - loss: 0.3803 - accuracy: 0.8617 - val_loss: 0.7416 - val_accuracy: 0.7298
Epoch 5/50
87/87 [==============================] - 0s 5ms/step - loss: 0.3471 - accuracy: 0.8725 - val_loss: 1.0191 - val_accuracy: 0.6662
Epoch 6/50
13/13 [==============================] - 0s 2ms/step - loss: 1.0875 - accuracy: 0.6026
Epoch 1/50
87/87 [==============================] - 2s 9ms/step - loss: 0.5254 - accuracy: 0.8097 - val_loss: 0.4438 - val_accuracy: 0.8353
Epoch 2/50
87/87 [==============================] - 0s 5ms/step - loss: 0.4714 - accuracy: 0.8277 - val_loss: 0.4931 - val_accuracy: 0.8020
Epoch 3/50
87/87 [==============================] - 0s 5ms/step - loss: 0.4122 - accuracy: 0.8490 - val_loss: 0.5509 - val_accuracy: 0.7803
Epoch 4/50
87/87 [==============================] - 0s 5ms/step - loss: 0

Epoch 2/50
87/87 [==============================] - 0s 5ms/step - loss: 0.2746 - accuracy: 0.9039 - val_loss: 0.3923 - val_accuracy: 0.8613
Epoch 3/50
87/87 [==============================] - 0s 5ms/step - loss: 0.2346 - accuracy: 0.9166 - val_loss: 0.4230 - val_accuracy: 0.8483
Epoch 4/50
87/87 [==============================] - 0s 5ms/step - loss: 0.2118 - accuracy: 0.9252 - val_loss: 0.5233 - val_accuracy: 0.8078
Epoch 5/50
87/87 [==============================] - 0s 5ms/step - loss: 0.1832 - accuracy: 0.9390 - val_loss: 0.6798 - val_accuracy: 0.7832
Epoch 6/50
13/13 [==============================] - 0s 2ms/step - loss: 1.4583 - accuracy: 0.5610
Epoch 1/50
87/87 [==============================] - 2s 9ms/step - loss: 0.3266 - accuracy: 0.8837 - val_loss: 0.2854 - val_accuracy: 0.9104
Epoch 2/50
87/87 [==============================] - 0s 5ms/step - loss: 0.2761 - accuracy: 0.9000 - val_loss: 0.3795 - val_accuracy: 0.8555
Epoch 3/50
87/87 [==============================] - 0s 5ms/ste

87/87 [==============================] - 0s 4ms/step - loss: 0.3972 - accuracy: 0.8522 - val_loss: 1.3355 - val_accuracy: 0.5642
Epoch 8/50
13/13 [==============================] - 0s 2ms/step - loss: 0.9930 - accuracy: 0.5922
Epoch 1/50
87/87 [==============================] - 1s 5ms/step - loss: 0.8204 - accuracy: 0.6844 - val_loss: 0.7232 - val_accuracy: 0.7327
Epoch 2/50
87/87 [==============================] - 0s 4ms/step - loss: 0.6987 - accuracy: 0.7299 - val_loss: 0.7414 - val_accuracy: 0.7298
Epoch 3/50
87/87 [==============================] - 0s 4ms/step - loss: 0.6155 - accuracy: 0.7692 - val_loss: 0.8034 - val_accuracy: 0.6835
Epoch 4/50
87/87 [==============================] - 0s 4ms/step - loss: 0.5228 - accuracy: 0.8072 - val_loss: 0.8530 - val_accuracy: 0.6922
Epoch 5/50
87/87 [==============================] - 0s 4ms/step - loss: 0.4379 - accuracy: 0.8375 - val_loss: 0.9414 - val_accuracy: 0.6850
Epoch 6/50
13/13 [==============================] - 0s 2ms/step - loss: 0

87/87 [==============================] - 0s 4ms/step - loss: 0.2596 - accuracy: 0.9187 - val_loss: 0.5821 - val_accuracy: 0.7746
Epoch 5/50
87/87 [==============================] - 0s 4ms/step - loss: 0.2266 - accuracy: 0.9278 - val_loss: 0.6533 - val_accuracy: 0.7673
Epoch 6/50
13/13 [==============================] - 0s 2ms/step - loss: 1.1224 - accuracy: 0.6286
Epoch 1/50
87/87 [==============================] - 1s 5ms/step - loss: 0.3856 - accuracy: 0.8562 - val_loss: 0.4435 - val_accuracy: 0.8557
Epoch 2/50
87/87 [==============================] - 0s 4ms/step - loss: 0.3027 - accuracy: 0.9025 - val_loss: 0.5401 - val_accuracy: 0.8153
Epoch 3/50
87/87 [==============================] - 0s 4ms/step - loss: 0.2690 - accuracy: 0.9053 - val_loss: 0.6034 - val_accuracy: 0.7980
Epoch 4/50
87/87 [==============================] - 0s 4ms/step - loss: 0.2264 - accuracy: 0.9227 - val_loss: 0.6902 - val_accuracy: 0.7677
Epoch 5/50
87/87 [==============================] - 0s 4ms/step - loss: 0

87/87 [==============================] - 0s 4ms/step - loss: 0.1430 - accuracy: 0.9588 - val_loss: 0.2700 - val_accuracy: 0.9118
Epoch 4/50
87/87 [==============================] - 0s 4ms/step - loss: 0.1136 - accuracy: 0.9686 - val_loss: 0.3278 - val_accuracy: 0.8815
Epoch 5/50
87/87 [==============================] - 0s 4ms/step - loss: 0.0967 - accuracy: 0.9718 - val_loss: 0.3786 - val_accuracy: 0.8757
Epoch 6/50
13/13 [==============================] - 0s 2ms/step - loss: 1.5047 - accuracy: 0.6234
Epoch 1/50
87/87 [==============================] - 1s 5ms/step - loss: 0.1999 - accuracy: 0.9354 - val_loss: 0.1538 - val_accuracy: 0.9509
Epoch 2/50
87/87 [==============================] - 0s 4ms/step - loss: 0.1529 - accuracy: 0.9541 - val_loss: 0.1921 - val_accuracy: 0.9321
Epoch 3/50
87/87 [==============================] - 0s 4ms/step - loss: 0.1262 - accuracy: 0.9632 - val_loss: 0.2328 - val_accuracy: 0.9162
Epoch 4/50
87/87 [==============================] - 0s 4ms/step - loss: 0

In [39]:
len(random_seeds)

5

In [40]:
rnn_results = results[model_names[0]]
lstm_results = results[model_names[1]]
rnn_results = results[model_names[2]]
rnn_results = results[model_names[3]]

In [41]:
df = pd.DataFrame(rnn_results)
df

,135,755,47,243,513
loss,0.952963,1.038754,1.224818,1.484107,1.831949
accuracy,0.622338,0.642078,0.620779,0.617662,0.615065
recall,0.625985,0.646267,0.624015,0.622920,0.617623
precision,0.621763,0.642767,0.621127,0.617522,0.615355
f1,0.621623,0.641860,0.619397,0.616554,0.613106


In [42]:
df = pd.DataFrame(lstm_results)
df

,135,755,47,243,513
loss,1.008590,1.134496,1.267444,1.444094,1.685610
accuracy,0.609870,0.597403,0.605195,0.600000,0.601039
recall,0.613412,0.600480,0.604851,0.600148,0.609185
precision,0.610138,0.596348,0.604974,0.599846,0.599856
f1,0.609598,0.596231,0.602570,0.598561,0.602257


## 3-3) 결과 도출

In [43]:
#데이터 프레임을 통한 시드별 결과 도출
df_data = []
for model_name in results:
    for seed, scores in results[model_name].items():
        df_data.append({
            'Model': model_name,
            'Seed': seed,
            'Loss': scores['loss'],
            'Accuracy': scores['accuracy'],
            'Recall':scores['recall'],
            'Precision' :scores['precision'],
            'F1': scores['f1']
        })

# 데이터프레임 생성
df = pd.DataFrame(df_data)
print(df)

     Model  Seed      Loss  Accuracy    Recall  Precision        F1
0      RNN   135  1.437677  0.420779  0.417651   0.417006  0.394490
1      RNN   755  1.674351  0.450390  0.444841   0.448444  0.443369
2      RNN    47  1.929820  0.483117  0.470458   0.481362  0.470750
3      RNN   243  2.380812  0.450390  0.439073   0.447874  0.440142
4      RNN   513  3.027368  0.446753  0.433266   0.443965  0.434872
5     LSTM   135  1.008590  0.609870  0.613412   0.610138  0.609598
6     LSTM   755  1.134496  0.597403  0.600480   0.596348  0.596231
7     LSTM    47  1.267444  0.605195  0.604851   0.604974  0.602570
8     LSTM   243  1.444094  0.600000  0.600148   0.599846  0.598561
9     LSTM   513  1.685610  0.601039  0.609185   0.599856  0.602257
10     GRU   135  1.034440  0.561558  0.560040   0.561314  0.549935
11     GRU   755  1.056630  0.603117  0.610745   0.602622  0.603056
12     GRU    47  1.231621  0.575065  0.571434   0.575611  0.569892
13     GRU   243  1.548044  0.568312  0.570318  

In [44]:
#시드 결과 평균
avg_df = []

for model_name in df['Model'].unique():
    average_loss = sum(df[df['Model'] == model_name]["Loss"].tolist()) / len(random_seeds)
    average_accuracy = sum(df[df['Model'] == model_name]["Accuracy"].tolist()) / len(random_seeds)
    average_recall = sum(df[df['Model'] == model_name]["Recall"].tolist()) / len(random_seeds)
    average_precision = sum(df[df['Model'] == model_name]["Precision"].tolist()) / len(random_seeds)
    average_f1 = sum(df[df['Model'] == model_name]["F1"].tolist()) / len(random_seeds)
    
    avg_df.append({
        'Model' : model_name,
        'average_loss' : average_loss,
        'average_accuracy':average_accuracy,
        'average_recall' : average_recall,
        'average_precision':average_precision,
        'average_f1':average_f1,
    })
    
# 데이터프레임 생성
df = pd.DataFrame(avg_df)
print(df)

    Model  average_loss  average_accuracy  average_recall  average_precision  \
0     RNN      2.090006          0.450286        0.441058           0.447730   
1    LSTM      1.308047          0.602701        0.605615           0.602233   
2     GRU      1.350605          0.576312        0.577796           0.576453   
3  1D CNN      1.306518          0.623584        0.627362           0.623707   

   average_f1  
0    0.436725  
1    0.601843  
2    0.572626  
3    0.622508  
